## Difference two DEMs

In [5]:
import numpy as np
import rasterio
from rasterio.coords import BoundingBox
from rasterio.mask import mask

import pandas as pd
import geopandas as gpd
import math

from pathlib import Path
import matplotlib.pyplot as plt

import os


In [34]:
# #Adjusted and integer aligned (interpolation done by HDR)
# #inputs
# dem1 = Path(r"C:\Users\jlogan\OneDrive - DOI\LRDP\LosPadresRes\bathy\ReprojectedBathyDougSmith\2016_correction\HDR_LP1m_NAD832011_depth_dependent_corrected_v2_integer_aligned.tif")
# dem2 = Path(r"C:\Users\jlogan\OneDrive - DOI\LRDP\LosPadresRes\bathy\ReprojectedBathyDougSmith\2017\lp17cube1m_NAD832011_integer_aligned.tif")
# #out
# outunclipped = Path(r"C:\Users\jlogan\OneDrive - DOI\LRDP\LosPadresRes\DoD\DoD_2017-HDR_DepCorrV2_script_generated.tif")
# outclipped = Path(r"C:\Users\jlogan\OneDrive - DOI\LRDP\LosPadresRes\DoD\DoD_2017-HDR_DepCorrV2_script_generated_clipped.tif")

# #Bathy pts adjusted, interpolated (tinned and tin_ras) by jlogan
# #inputs
# dem1 = Path(r"C:\Users\jlogan\OneDrive - DOI\LRDP\LosPadresRes\bathy\HDR_2016_reinterpolate\tinras\HDR_reinterp_adj_1m.tif")
# dem2 = Path(r"C:\Users\jlogan\OneDrive - DOI\LRDP\LosPadresRes\bathy\ReprojectedBathyDougSmith\2017\lp17cube1m_NAD832011_integer_aligned.tif")
# #out
# outunclipped = Path(r"C:\Users\jlogan\OneDrive - DOI\LRDP\LosPadresRes\DoD\DoD_2017-HDR_Reinterp_script_gen.tif")
# outclipped = Path(r"C:\Users\jlogan\OneDrive - DOI\LRDP\LosPadresRes\DoD\DoD_2017-HDR_Reinterp_script_gen_clipped.tif")

# #Bathy pts adjusted, interpolated (tinned and tin_ras) by jlogan (5m cell size)
# #inputs
# dem1 = Path(r"C:\Users\jlogan\OneDrive - DOI\LRDP\LosPadresRes\bathy\HDR_2016_reinterpolate\tinras\HDR_reinterp_adj_5m.tif")
# dem2 = Path(r"C:\Users\jlogan\OneDrive - DOI\LRDP\LosPadresRes\bathy\ReprojectedBathyDougSmith\2017\lp17cube1m_NAD832011_int_al_res5m.tif")
# #out
# outunclipped = Path(r"C:\Users\jlogan\OneDrive - DOI\LRDP\LosPadresRes\DoD\DoD_2017-HDR_Reinterp_script_gen_5m.tif")
# outclipped = Path(r"C:\Users\jlogan\OneDrive - DOI\LRDP\LosPadresRes\DoD\DoD_2017-HDR_Reinterp_script_gen_clipped_5m.tif")

# #Bathy pts adjusted, interpolated (tinned and tin_ras) by jlogan (2m cell size)
# #inputs
# dem1 = Path(r"C:\Users\jlogan\OneDrive - DOI\LRDP\LosPadresRes\bathy\HDR_2016_reinterpolate\tinras\HDR_reinterp_adj_2m.tif")
# dem2 = Path(r"C:\Users\jlogan\OneDrive - DOI\LRDP\LosPadresRes\bathy\ReprojectedBathyDougSmith\2017\lp17cube1m_NAD832011_int_al_res2m.tif")
# #out
# outunclipped = Path(r"C:\Users\jlogan\OneDrive - DOI\LRDP\LosPadresRes\DoD\DoD_2017-HDR_Reinterp_script_gen_2m.tif")
# outclipped = Path(r"C:\Users\jlogan\OneDrive - DOI\LRDP\LosPadresRes\DoD\DoD_2017-HDR_Reinterp_script_gen_clipped_2m.tif")

#Bathy pts adjusted, interpolated (tinned and tin_ras) by jlogan (2m cell size)
#inputs
dem1 = Path(r"C:\Users\jlogan\OneDrive - DOI\LRDP\LosPadresRes\bathy\HDR_2016_reinterpolate\tinras\HDR_reinterp_adj_breaklines_1m.tif")
dem2 = Path(r"C:\Users\jlogan\OneDrive - DOI\LRDP\LosPadresRes\bathy\ReprojectedBathyDougSmith\2017\lp17cube1m_NAD832011_integer_aligned.tif")
#out
outunclipped = Path(r"C:\Users\jlogan\OneDrive - DOI\LRDP\LosPadresRes\DoD\DoD_2017-HDR_Reinterp_breaklines_script_gen_1m.tif")
outclipped = Path(r"C:\Users\jlogan\OneDrive - DOI\LRDP\LosPadresRes\DoD\DoD_2017-HDR_Reinterp_breaklines_script_gen_clipped_1m.tif")

#Raster cell size
cellsize = 1

#DoD clip
clip_shp = Path(r"C:\Users\jlogan\OneDrive - DOI\LRDP\LosPadresRes\DoD\shp\2016_hull_poly.shp")

In [36]:
#DEM difference
#with ChatGPT assitance
# Open the input raster files using rasterio's context manager
with rasterio.open(dem1) as dataset1, rasterio.open(dem2) as dataset2:
    # Read the raster data as NumPy arrays
    raster_data1 = dataset1.read(1)
    raster_data2 = dataset2.read(1)

    # Retrieve the affine transformation matrices
    transform1 = dataset1.transform
    transform2 = dataset2.transform

    # Determine the common extent
    bounds1 = dataset1.bounds
    bounds2 = dataset2.bounds
    intersection_bounds = BoundingBox(
        max(bounds1.left, bounds2.left),
        max(bounds1.bottom, bounds2.bottom),
        min(bounds1.right, bounds2.right),
        min(bounds1.top, bounds2.top)
        )

   # Update the transformation matrices to the common extent
    window1 = dataset1.window(*intersection_bounds)
    window2 = dataset2.window(*intersection_bounds)
    transform1 = rasterio.windows.transform(window1, transform1)
    transform2 = rasterio.windows.transform(window2, transform2)

    # Read the subset of raster data within the common extent
    raster_data1 = dataset1.read(1, window=window1)
    raster_data2 = dataset2.read(1, window=window2)

    # Compute the difference between the two rasters
    difference = raster_data2 - raster_data1
    
    # Set nodata values in the difference array
    nodata_mask = np.logical_or(raster_data1 == dataset1.nodata, raster_data2 == dataset2.nodata)
    difference[nodata_mask] = dataset1.nodata

    # Retrieve the metadata from one of the input rasters
    metadata = dataset1.meta

    # Update the metadata to reflect the difference raster
    metadata.update(
        count=1,
        height=difference.shape[0],
        width=difference.shape[1],
        transform=transform1,
        nodata=dataset1.nodata
    )

    # Create a new raster file for the difference
    output_file = outunclipped
    with rasterio.open(str(output_file), 'w', **metadata) as output_dataset:
        output_dataset.write(difference, 1)

print("Raster difference complete.")


Raster difference complete.


In [37]:
#clip

# Read the shapefile using geopandas
shapefile = gpd.read_file(clip_shp)

# Read the raster using rasterio
with rasterio.open(outunclipped) as src:
    # Clip the raster using the shapefile geometry
    clipped_raster, clipped_transform = mask(src, shapefile.geometry, crop=True)
    clipped_meta = src.meta

# Update the metadata of the clipped raster
clipped_meta.update({
    'height': clipped_raster.shape[1],
    'width': clipped_raster.shape[2],
    'transform': clipped_transform
})

# Write the clipped raster to the output path
with rasterio.open(outclipped, 'w', **clipped_meta) as dst:
    dst.write(clipped_raster)

#delete unclipped
os.remove(outunclipped)

print('Raster clipping complete, deleteing unclipped raster. Clipped raster saved to:', output_file)


Raster clipping complete, deleteing unclipped raster. Clipped raster saved to: C:\Users\jlogan\OneDrive - DOI\LRDP\LosPadresRes\DoD\DoD_2017-HDR_Reinterp_breaklines_script_gen_1m.tif


In [43]:
#Calc sum
with rasterio.open(outclipped) as src:
    # Read the raster data as NumPy arrays
    data = src.read(1, masked=True)

    masked_raster = np.ma.masked_equal(data, src.nodata)
    masked_sum = np.sum(masked_raster)
    
    vol = np.sum(masked_raster) * cellsize * cellsize
    area = masked_raster.count() * cellsize * cellsize
    
    print(f'Volume: {vol} m^3\nArea: {area} m^2')

Volume: 39044.01953125 m^3
Area: 161587 m^2
